# Getting Data from DFO Water Levels Service

Example code to access the DFO Water Levels web service,
and information about what data are available there.

Service description: http://www.tides.gc.ca/eng/info/WebServicesWLD

Technical specification (PDF): http://www.tides.gc.ca/docs/Specifications%20-%20Spine%20observation%20and%20predictions%202.0.3%28en%29.pdf

The service uses the [SOAP](https://en.wikipedia.org/wiki/SOAP) protocol.
The are several Python libraries for working with SOAP services.
The one used here is [suds-jurko](https://bitbucket.org/jurko/suds),
a Python 3 port of the (apparently) no longer maintained [suds](http://fedorahosted.org/suds) library.
Docs for `suds` and `suds-jurko` are at https://fedorahosted.org/suds/wiki/Documentation.

You can install `suds-jurko` from the IOOS conda channel with:

    $ conda install -c https://conda.anaconda.org/ioos suds-jurko

or with `pip`:


    $ pip install suds-jurko


In [1]:
from suds.client import Client

The service has 3 entry point URLs:

* SPINE (St. Lawerence water level forecast & interpolation): https://ws-shc.qc.dfo-mpo.gc.ca/spine 
* observations: https://ws-shc.qc.dfo-mpo.gc.ca/observations
* Tide table predictions: https://ws-shc.qc.dfo-mpo.gc.ca/predictions

We're interesting in the observations.

In [2]:
obs_url = 'https://ws-shc.qc.dfo-mpo.gc.ca/observations'

In [9]:
client = Client(obs_url + '?wsdl')

In [11]:
print(client)


Suds ( https://fedorahosted.org/suds/ )  version: 0.6

Service ( ObservationsService ) tns="http://client.ws.shc.gc.ca"
   Prefixes (3)
      ns0 = "http://client.ws.shc.gc.ca"
      ns1 = "http://schemas.xmlsoap.org/soap/encoding/"
      ns2 = "http://wds.dfo.gc.ca"
   Ports (1):
      (observations)
         Methods (13):
            getBoundaryDate()
            getBoundaryDepth()
            getBoundarySpatial()
            getDataInfo()
            getInfo()
            getLocalizedString(xs:string in0, ArrayOf_xsd_anyType in1)
            getMetadata()
            getMetadataInfo()
            getName()
            getResourceBundle()
            getStatus()
            getVersion()
            search(xs:string dataName, xs:double latitudeMin, xs:double latitudeMax, xs:double longitudeMin, xs:double longitudeMax, xs:double depthMin, xs:double depthMax, xs:string dateMin, xs:string dateMax, xs:int start, xs:int sizeMax, xs:boolean metadata, xs:string metadataSelection, xs:string 

In [10]:
client.service.getStatus()

(Status){
   message = "Ok"
   status = "ok"
 }

In [12]:
client.service.getVersion()

1.0.0

In [13]:
client.service.getInfo()

This Web Service gives the access to the observations at the SINECO stations

In [14]:
client.service.getDataInfo()

[(Metadata){
    name = "wl"
    value = "Water levels at the SINECO stations"
  }, (Metadata){
    name = "sal"
    value = "Salinity at the SINECO stations"
  }, (Metadata){
    name = "temp"
    value = "Temperature at the SINECO stations"
  }, (Metadata){
    name = "atm_pres"
    value = "Atmospheric pressure at the SINECO stations"
  }]

In [16]:
metadata = client.service.getMetadata()
print(metadata)

[(Metadata){
   name = "station_id_list"
   value = "01970,02330,02780,02985,03057,03071,03075,03100,03110,03248,03280,03300,03335,03345,03353,03360,03365,03424,07277,07594,07654,07735,07786,07917,08408,09338,09341,09348,09354,15330,15520,15540,15660,15780,15930,15975,16005"
 }, (Metadata){
   name = "station_id_position"
   value = "[01970,47.378861,-61.857293][02330,48.997,-64.3805][02780,50.194833,-66.376833][02985,48.478333,-68.513667][03057,47.448833,-70.3655][03071,47.161833,-70.607667][03075,47.0895,-70.710833][03100,46.9965,-70.808167][03110,46.858167,-71.003333][03248,46.811111,-71.201944][03280,46.6965,-71.572833][03300,46.68116667,-71.877167][03335,46.561,-72.105833][03345,46.500333,-72.245833][03353,46.400333,-72.3795][03360,46.3405,-72.539167][03365,46.2725,-72.619333][03424,48.126333,-69.72975][07277,48.653601,-123.451646][07594,49.105896,-123.303368][07654,49.200077,-122.910377][07735,49.289554,-123.107339][07786,49.340556,-123.231978][07917,49.162788,-123.923523][08408,

In [17]:
len(metadata)

11

In [20]:
stn_names = metadata[7]

In [56]:
[stn for stn in stn_names.value[1:-1].split('][') if '(BC)' in stn]

['07277;;Patricia Bay (BC)',
 '07594;;SandHeads (BC)',
 '07654;;New Westminster (BC)',
 '07735;;Vancouver (BC)',
 '07786;;Sandy Cove (BC)',
 '07917;;Port of Nanaimo (BC)',
 '09338;;PRPA - Aero Trading (BC)',
 '09341;;Porpoise Channel East (BC)',
 '09348;;PRPA - Fairview Terminal (BC)',
 '09354;;Prince Rupert (BC)']

search(xs:string dataName, xs:double latitudeMin, xs:double latitudeMax, xs:double longitudeMin, xs:double longitudeMax, xs:double depthMin, xs:double depthMax, xs:string dateMin, xs:string dateMax, xs:int start, xs:int sizeMax, xs:boolean metadata, xs:string metadataSelection, xs:string order)

In [36]:
client.service.search('wl', -90, 90, -180, 180, 0, 0, '2015-11-03 17:00:00', '2015-11-03 18:00:00', 1, 1, True, 'station_id=07735', 'asc')

(ResultSet){
   boundaryDate = 
      (BoundaryDate){
         max = "2015-11-03 17:00:00"
         min = "2015-11-03 17:00:00"
      }
   boundaryDepth = 
      (BoundaryDepth){
         max = 0.0
         min = 0.0
      }
   boundarySpatial = 
      (BoundarySpatial){
         latitude = 
            (RealBoundary){
               max = 49.289554
               min = 49.289554
            }
         longitude = 
            (RealBoundary){
               max = -123.107339
               min = -123.107339
            }
      }
   boundaryValue = 
      (StringBoundary){
         max = "3.54"
         min = "3.54"
      }
   data[] = 
      (Data){
         boundaryDate = 
            (BoundaryDate){
               max = "2015-11-03 17:00:00"
               min = "2015-11-03 17:00:00"
            }
         boundaryDepth = 
            (BoundaryDepth){
               max = 0.0
               min = 0.0
            }
         metadata[] = 
            (Metadata){
               name = "

In [45]:
data = client.service.search('wl', -90, 90, -180, 180, 0, 0, '2015-11-03 17:00:00', '2015-11-03 18:00:00', 1, 1, True, 'station_id=07735', 'asc')
print(data)

(ResultSet){
   boundaryDate = 
      (BoundaryDate){
         max = "2015-11-03 17:00:00"
         min = "2015-11-03 17:00:00"
      }
   boundaryDepth = 
      (BoundaryDepth){
         max = 0.0
         min = 0.0
      }
   boundarySpatial = 
      (BoundarySpatial){
         latitude = 
            (RealBoundary){
               max = 49.289554
               min = 49.289554
            }
         longitude = 
            (RealBoundary){
               max = -123.107339
               min = -123.107339
            }
      }
   boundaryValue = 
      (StringBoundary){
         max = "3.54"
         min = "3.54"
      }
   data[] = 
      (Data){
         boundaryDate = 
            (BoundaryDate){
               max = "2015-11-03 17:00:00"
               min = "2015-11-03 17:00:00"
            }
         boundaryDepth = 
            (BoundaryDepth){
               max = 0.0
               min = 0.0
            }
         metadata[] = 
            (Metadata){
               name = "

In [46]:
type(data)

suds.sudsobject.ResultSet

In [47]:
data.data

[(Data){
    boundaryDate = 
       (BoundaryDate){
          max = "2015-11-03 17:00:00"
          min = "2015-11-03 17:00:00"
       }
    boundaryDepth = 
       (BoundaryDepth){
          max = 0.0
          min = 0.0
       }
    metadata[] = 
       (Metadata){
          name = "station_id"
          value = "07735"
       },
       (Metadata){
          name = "station_name"
          value = "Vancouver (BC)"
       },
       (Metadata){
          name = "vl"
          value = "2"
       },
    spatialCoordinates[] = 
       (SpatialCoordinate){
          latitude = 49.289554
          longitude = -123.107339
       },
    value = "3.54"
  }]

In [48]:
d = data.data

In [49]:
type(d)

list

In [50]:
len(d)

1

In [51]:
d0 = d[0]

In [52]:
type(d0)

suds.sudsobject.Data

In [53]:
d0.value

3.54